In [1]:
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point

block = pd.read_csv('..//data//census//tl_2010_06075_tabblock10.csv')

# Create list and path to shapefile
block_list = []
lon, lat = [], []
shape_loc = 'Land Use//geo_export_d1237e9c-8636-4824-8637-a2a9e5f4b343.shp'
c_file = fiona.open("..//data//sf_open//{}".format(shape_loc))
pol = c_file.next()

for i in range(0,155467):
    
    geom = shape(pol['geometry'])
    block_list.append(dict(pol['properties']))
    try:
        lon.append(geom.exterior.centroid.xy[0][0])
        lat.append(geom.exterior.centroid.xy[1][0])
    except:
        lon.append(np.nan)
        lat.append(np.nan)
    pol = c_file.next()
    
    
c_file.close()
    
block_df = pd.DataFrame(block_list)
block_df['longitude'] = lon
block_df['latitude'] = lat
#block_df.to_csv('..//data//created//all_blocks.csv', index=False)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.


In [2]:
sq_ft_use = pd.pivot_table(block_df,
                           index=['block_num',
                                  'latitude',
                                  'longitude'],
                           columns='landuse',
                           values = 'bldgsqft').reset_index()

sq_ft_use = sq_ft_use.fillna(0)

sq_ft_use = sq_ft_use.groupby('block_num').mean().reset_index()

sq_ft_use = sq_ft_use.loc[:,[
    
    'block_num',
    'RESIDENT',
    'MIXRES',
    'RETAIL/ENT',
    'CIE',
    'MED',
    'MIPS',
    'PDR',
    'VISITOR',
    'VACANT',
    #'ROW',
    #'OPENSPACE',
    'latitude',
    'longitude'
]]

sq_ft_use.columns = [
    
    'block_num', 
    'res_sqft',
    'mixres_sqft',
    'retail_sqft',
    'cie_sqft',
    'medical_sqft',
    'office_sqft',
    'indus_sqft',
    'hotel_sqft',
    'vacant_sqft',
    #'row_sqft',
    #'openspace_sqft',
    'latitude',
    'longitude'
]

In [13]:
block_id = []
shape_loc = 'Census 2000_ Blocks for San Francisco (no water)//geo_export_10e804e3-5dd0-4986-9466-956443c0fd64.shp'
for index, row in sq_ft_use.iterrows():
    
    # open the shapefile and select the first record
    c_file = fiona.open("..//data//sf_open//{}".format(shape_loc))
    pol = c_file.next()
    geom = shape(pol['geometry'])

    # create the first test point
    point_test = Point((row['longitude'],row['latitude']))
    
    try:
        
        # iterate through all the blocks until a match is found
        while geom.contains(point_test) == False:
            pol = c_file.next()
            geom = shape(pol['geometry'])
        
        # append the block id to the empty list
        block_id.append(block['GEOID10'][int(pol['id'])])
        
    except:
        block_id.append(np.nan)
        
    c_file.close()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.


In [14]:
sq_ft_use['block_id'] = block_id

In [18]:
sq_ft_use.to_csv('..//data//created//land_use_blockid.csv', index=False)